# Syft Kerasを用いたプライバシーに配慮した予測

# ステップ1 : 公開データでの学習
チュートリアルへようこそ！今回のノートブックでは、プライバシーに配慮した予測に関する手法について学習します。プライバシーに配慮した予測とは、データがプロセス全体を通して常に暗号化されることを意味します。ユーザーは生データを共有せず、暗号化された（つまり、秘密の共有）データのみを共有します。これらのプライバシーに配慮した予測を提供するために、Syft Kerasは内部で[TF Encrypted]（https://github.com/tf-encrypted/tf-encrypted）と呼ばれるライブラリを使用します。 TF Encryptedは、最先端の暗号技術と機械学習技術を組み合わせていますが、暗号技術について考慮する必要はなく、機械学習アプリケーションを利用できます。

次の3つの手順でプライベート予測の配信を開始できます。
-**ステップ1 **：通常のKerasを使用するモデルの学習
-**ステップ2 **：機械学習モデル（サーバー）を保護した状態での提供。
-**ステップ3 **：保護されたモデルにクエリを実行する、プライバシーに配慮した予測の受信(クライアント)。

では、これらの3つの手順を実行して、ユーザーのプライバシーやモデルのセキュリティを無駄にすることなく、効果のある機械学習サービスを展開できるようにします。

Authors:
- Jason Mancuso - Twitter: [@jvmancuso](https://twitter.com/jvmancuso)
- Yann Dupis - Twitter: [@YannDupis](https://twitter.com/YannDupis)
- Morten Dahl - Twitter: [@mortendahlcs](https://github.com/mortendahlcs)

On behalf of:
- Dropout Labs - Twitter: [@dropoutlabs](https://twitter.com/dropoutlabs)
- TF Encrypted - Twitter: [@tf_encrypted](https://twitter.com/tf_encrypted)

## Keras使用するモデルの学習

 [Keras](https://keras.io/) を使用するモデルの学習
プロジェクトでプライバシーを保護する機械学習手法を使用するために、新しい機械学習フレームワークを学習する必要はありません。 Kerasの基本的な知識がある場合は、Syft Kerasでこれらの手法を使用して開始できます。以前にKerasを使用したことがない場合は、[Keras documentation](https://keras.io)のドキュメントで詳細を確認できます。

プライバシーに配慮した予測の前に、最初のステップはモデルを通常のKerasでトレーニングする点です。例として、手書きの数字を分類するモデルをトレーニングします。このモデルをトレーニングするには、標準の[MNIST dataset](http://yann.lecun.com/exdb/mnist/)データセットを使用します。

参照Kerasリポジトリから[this example](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) を用います。分類モデルをトレーニングするには、以下のセルを実行するだけです。

In [ ]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# 入力の次元数
img_rows, img_cols = 28, 28

# 学習用とテスト用のデータに分割
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# クラスラベルの変換
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## 学習済みモデルの重みの保存

すごい！モデルは訓練済みです。`model.save()`でモデルの重みを保存しましょう。次のノートブックでは、この学習済みモデルの重みをSyft Kerasにロードして、プライベート予測の提供を開始します。

In [ ]:
model.save('short-conv-mnist.h5')